tutorial at:

https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/tour-of-beam/reading-and-writing-data.ipynb#scrollTo=sQUUi4H9s-g2


make files

In [ ]:
%%writefile data/my-text-file-1.txt
This is just a plain text file, UTF-8 strings are allowed 🎉.
Each line in the file is one element in the PCollection.

In [ ]:
%%writefile data/my-text-file-2.txt
There are no guarantees on the order of the elements.
ฅ^•ﻌ•^ฅ

Read files, each line is an element in th p collection

In [ ]:
import apache_beam as beam
input_files = 'data/*.txt'
with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Read files' >> beam.io.ReadFromText(input_files)
        | 'Print contents' >> beam.Map(print)
    )

write files

In [ ]:
output_file_name_prefix = 'outputs/file'

lines_to_write = [
    'Each element must be a string.',
    'It writes one element per line.',
    'There are no guarantees on the line order.',
    'The data might be written into multiple files.',
]

with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Create file lines' >> beam.Create(lines_to_write)
        | 'Write to files' >> beam.io.WriteToText(
            output_file_name_prefix,
            file_name_suffix='.txt')
    )

In [ ]:
# Lets look at the output files and contents.
!head outputs/file*.txt

Reading from an iterable

The easiest way to create elements is using FlatMap.
A common way is having a generator function. This could take an input and expand it into a large amount of elements. The nice thing about generators is that they don't have to fit everything into memory like a list, they simply yield elements as they process them.
For example, let's define a generator called count, that yields the numbers from 0 to n. We use Create for the initial n value(s) and then exapand them with FlatMap.

In [ ]:
import apache_beam as beam

def count(n):
  for i in range(n):
    yield i

n = 5
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create inputs' >> beam.Create([n])
      | 'Generate elements' >> beam.FlatMap(count)
      | 'Print elements' >> beam.Map(print)
  )

Creating an input transform

For a nicer interface, we could abstract the Create and the FlatMap into a custom PTransform. This would give a more intuitive way to use it, while hiding the inner workings.
We create a new class that inherits from beam.PTransform. Any input from the generator function, like n, becomes a class field. The generator function itself would now become a staticmethod. And we can hide the Create and FlatMap in the expand method.
Now we can use our transform in a more intuitive way, just like ReadFromText.

In [ ]:
import apache_beam as beam

class Count(beam.PTransform):
  def __init__(self, n):
    self.n = n

  @staticmethod
  def count(n):
    for i in range(n):
      yield i

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create inputs' >> beam.Create([self.n])
        | 'Generate elements' >> beam.FlatMap(Count.count)
    )

n = 3
with beam.Pipeline() as pipeline:
  (
      pipeline
      | f'Count to {n}' >> Count(n)
      | 'Print elements' >> beam.Map(print)
  )

Example: Reading CSV files

Lets say we want to read CSV files to get elements as Python dictionaries. We like how ReadFromText expands a file pattern, but we might want to allow for multiple patterns as well.
We create a ReadCsvFiles transform, which takes a list of file_patterns as input. It expands all the glob patterns, and then, for each file name it reads each row as a dict using the csv.DictReader module.

In [ ]:
%%writefile data/penguins.csv
species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
1,0.5236363636363636,0.5714285714285713,0.3389830508474576,0.2222222222222222
1,0.6509090909090909,0.7619047619047619,0.4067796610169492,0.3333333333333333
2,0.509090909090909,0.011904761904761862,0.6610169491525424,0.5
2,0.6509090909090909,0.38095238095238104,0.9830508474576272,0.8333333333333334

In [ ]:
import apache_beam as beam
import csv
import glob

class ReadCsvFiles(beam.PTransform):
  def __init__(self, file_patterns):
    self.file_patterns = file_patterns

  @staticmethod
  def read_csv_lines(file_name):
    with open(file_name, 'r') as f:
      for row in csv.DictReader(f):
        yield dict(row)

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create file patterns' >> beam.Create(self.file_patterns)
        | 'Expand file patterns' >> beam.FlatMap(glob.glob)
        | 'Read CSV lines' >> beam.FlatMap(self.read_csv_lines)
    )

input_patterns = ['data/*.csv']
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read CSV files' >> ReadCsvFiles(input_patterns)
      | 'Print elements' >> beam.Map(print)
  )

In [ ]:
import apache_beam as beam
import csv
import glob

def read_csv_lines(file_name):
    with open(file_name, 'r') as f:
        for row in csv.DictReader(f):
            yield dict(row)

input_patterns = ['data/*.csv']

with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Create file patterns' >> beam.Create(input_patterns)
      | 'Expand file patterns' >> beam.FlatMap(glob.glob)
      | 'Read CSV lines' >> beam.FlatMap(read_csv_lines)
      | 'Print elements' >> beam.Map(print)
  )